# How to update graph state from tools

<div class="admonition tip">
    <p class="admonition-title">Prerequisites</p>
    <p>
        This guide assumes familiarity with the following:
        <ul>
            <li>            
                <a href="https://langchain-ai.github.io/langgraphjs/concepts/low_level/#command">
                    Command
                </a>
            </li>
        </ul>
    </p>
</div>

A common use case is updating graph state from inside a tool. For example, in a customer support application you might want to look up customer account number or ID in the beginning of the conversation. To update the graph state from the tool, you can return a [`Command`](https://langchain-ai.github.io/langgraphjs/concepts/low_level/#command) object from the tool:

```ts
import { tool } from "@langchain/core/tools";

const lookupUserInfo = tool(async (input, config) => {
  const userInfo = getUserInfo(config);
  return new Command({
    // update state keys
    update: {
      user_info: userInfo,
      messages: [
        new ToolMessage({
          content: "Successfully looked up user information",
          tool_call_id: config.toolCall.id,
        }),
      ],
    },
  });
}, {
  name: "lookup_user_info",
  description: "Use this to look up user information to better assist them with their questions.",
  schema: z.object(...)
});
```
    
<div class="admonition note">
    <p class="admonition-title">Important</p>
    <p>
      If you want to use tools that return <code>Command</code> instances and update graph state, you can either use prebuilt <a href="https://langchain-ai.github.io/langgraphjs/reference/functions/langgraph_prebuilt.createReactAgent.html"><code>createReactAgent</code></a> / <a href="https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html"><code>ToolNode</code></a> components, or implement your own tool-executing node that identifies <code>Command</code> objects returned by your tools and returns a mixed array of traditional state updates and <code>Commands</code>.
      <br />
      See [this section](#custom-components) for an example.
    </p>
</div>

This guide shows how you can do this using LangGraph's prebuilt components ([`createReactAgent`](https://langchain-ai.github.io/langgraphjs/reference/functions/langgraph_prebuilt.createReactAgent.html) and [`ToolNode`](https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)).

<div class="admonition tip">
    <p class="admonition-title">Compatibility</p>
    <p>
        This guide requires <code>@langchain/langgraph>=0.2.33</code> and <code>@langchain/core@0.3.23</code>. For help upgrading, see <a href="/langgraphjs/how-tos/manage-ecosystem-dependencies/">this guide</a>.
    </p>
</div>

## Setup

Install the following to run this guide:

```bash
npm install @langchain/langgraph @langchain/openai @langchain/core
```

Next, configure your environment to connect to your model provider.

```bash
export OPENAI_API_KEY=your-api-key
```

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

Let's create a simple ReAct style agent that can look up user information and personalize the response based on the user info.

## Define tool

First, let's define the tool that we'll be using to look up user information. We'll use a naive implementation that simply looks user information up using a dictionary:

In [1]:
const USER_ID_TO_USER_INFO = {
  abc123: {
    user_id: "abc123",
    name: "Bob Dylan",
    location: "New York, NY",
  },
  zyx987: {
    user_id: "zyx987",
    name: "Taylor Swift",
    location: "Beverly Hills, CA",
  },
};

In [2]:
import { Annotation, Command, MessagesAnnotation } from "@langchain/langgraph";
import { tool } from "@langchain/core/tools";

import { z } from "zod";

const StateAnnotation = Annotation.Root({
  ...MessagesAnnotation.spec,
  // user provided
  lastName: Annotation<string>,
  // updated by the tool
  userInfo: Annotation<Record<string, any>>,
});

const lookupUserInfo = tool(async (_, config) => {
  const userId = config.configurable?.user_id;
  if (userId === undefined) {
    throw new Error("Please provide a user id in config.configurable");
  }
  if (USER_ID_TO_USER_INFO[userId] === undefined) {
    throw new Error(`User "${userId}" not found`);
  }
  // Populated when a tool is called with a tool call from a model as input
  const toolCallId = config.toolCall.id;
  return new Command({
    update: {
      // update the state keys
      userInfo: USER_ID_TO_USER_INFO[userId],
      // update the message history
      messages: [
        {
          role: "tool",
          content: "Successfully looked up user information",
          tool_call_id: toolCallId,
        },
      ],
    },
  })
}, {
  name: "lookup_user_info",
  description: "Always use this to look up information about the user to better assist them with their questions.",
  schema: z.object({}),
});

## Define prompt

Let's now add personalization: we'll respond differently to the user based on the state values AFTER the state has been updated from the tool. To achieve this, let's define a function that will dynamically construct the system prompt based on the graph state. It will be called ever time the LLM is called and the function output will be passed to the LLM:

In [3]:
const stateModifier = (state: typeof StateAnnotation.State) => {
  const userInfo = state.userInfo;
  if (userInfo == null) {
    return state.messages;
  }
  const systemMessage = `User name is ${userInfo.name}. User lives in ${userInfo.location}`;
  return [{
    role: "system",
    content: systemMessage,
  }, ...state.messages];
};

## Define graph

Finally, let's combine this into a single graph using the prebuilt `createReactAgent` and the components we declared earlier:

In [5]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
  model: "gpt-4o",
});

const agent = createReactAgent({
  llm: model,
  tools: [lookupUserInfo],
  stateSchema: StateAnnotation,
  stateModifier: stateModifier,
})

## Use it!

Let's now try running our agent. We'll need to provide user ID in the config so that our tool knows what information to look up:

In [6]:
const stream = await agent.stream({
  messages: [{
    role: "user",
    content: "hi, what should i do this weekend?",
  }],
  
}, {
  // provide user ID in the config
  configurable: { user_id: "abc123" }
});

for await (const chunk of stream) {
  console.log(chunk);
}

{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-AdmOZdrZy3aUgNimCIjq8ZW5js6ln",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_kLXWJYbabxWpj7vykXD6ZMx0",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "promptTokens": 59,
            "completionTokens": 11,
            "totalTokens": 70
          },
          "finish_reason": "tool_calls",
          "usage": {
            "prompt_tokens": 59,
            "completion_tokens": 11,
            "total_tokens": 70,
            "prompt_tokens_details": {
              "cached_tokens": 0,
              "audio_tokens": 0
            },
            "completion_tokens_details": {
              "reasoning_tokens": 0,
              "audio_tokens": 0,
              "accepted_prediction_tokens": 0,
              "rejected_predic

We can see that the model correctly recommended some New York activities for Bob Dylan! Let's try getting recommendations for Taylor Swift:

In [7]:
const taylorStream = await agent.stream({
  messages: [{
    role: "user",
    content: "hi, what should i do this weekend?",
  }],
  
}, {
  // provide user ID in the config
  configurable: { user_id: "zyx987" }
});

for await (const chunk of taylorStream) {
  console.log(chunk);
}

{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-AdmQGANyXPTAkMnQ86hGWB5XY5hGL",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_IvyfreezvohjGgUx9DrwfS5O",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "promptTokens": 59,
            "completionTokens": 11,
            "totalTokens": 70
          },
          "finish_reason": "tool_calls",
          "usage": {
            "prompt_tokens": 59,
            "completion_tokens": 11,
            "total_tokens": 70,
            "prompt_tokens_details": {
              "cached_tokens": 0,
              "audio_tokens": 0
            },
            "completion_tokens_details": {
              "reasoning_tokens": 0,
              "audio_tokens": 0,
              "accepted_prediction_tokens": 0,
              "rejected_predic

## Custom components

If you do not wish to use prebuilt components, you will need to have special logic in your custom tool executor to handle commands. Here's an example:

In [4]:
import {
  MessagesAnnotation,
  isCommand,
  Command,
  StateGraph,
} from "@langchain/langgraph";
import { tool } from "@langchain/core/tools";
import { isAIMessage } from "@langchain/core/messages";

import { z } from "zod";

const myTool = tool(async () => {
  return new Command({
    update: {
      messages: [
        {
          role: "assistant",
          content: "hi there!",
          name: "Greeter",
        }
      ],
    },
  });
}, {
  name: "greeting",
  description: "Updates the current state with a greeting",
  schema: z.object({}),
});

const toolExecutor = async (state: typeof MessagesAnnotation.State) => {
  const message = state.messages.at(-1);
  if (!isAIMessage(message) || message.tool_calls === undefined || message.tool_calls.length === 0) {
    throw new Error("Most recent message must be an AIMessage with a tool call.")
  }
  const outputs = await Promise.all(
    message.tool_calls.map(async (toolCall) => {
      // Using a single tool for simplicity, would need to select tools by toolCall.name
      // in practice.
      const toolResult = await myTool.invoke(toolCall);
      return toolResult;
    })
  );
  // Handle mixed Command and non-Command outputs
  const combinedOutputs = outputs.map((output) => {
    if (isCommand(output)) {
      return output;
    }
    // Tool invocation result is a ToolMessage, return a normal state update
    return { messages: [output] };
  });
  // Return an array of values instead of an object
  return combinedOutputs;
};

// Simple one node graph
const customGraph = new StateGraph(MessagesAnnotation)
  .addNode("runTools", toolExecutor)
  .addEdge("__start__", "runTools")
  .compile();
  
await customGraph.invoke({
  messages: [{
    role: "user",
    content: "how are you?",
  }, {
    role: "assistant",
    content: "Let me call the greeting tool and find out!",
    tool_calls: [{
      id: "123",
      args: {},
      name: "greeting",
    }],
  }],
});

{
  messages: [
    HumanMessage {
      "id": "801308df-c702-49f4-99c1-da4116f6bbc8",
      "content": "how are you?",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "8ea07329-a73a-4de4-a4d4-4453fbef32e0",
      "content": "Let me call the greeting tool and find out!",
      "additional_kwargs": {},
      "response_metadata": {},
      "tool_calls": [
        {
          "id": "123",
          "args": {},
          "name": "greeting"
        }
      ],
      "invalid_tool_calls": []
    },
    AIMessage {
      "id": "4ecba93a-77c0-44a6-8dc9-8b27d9615c15",
      "content": "hi there!",
      "name": "Greeter",
      "additional_kwargs": {},
      "response_metadata": {},
      "tool_calls": [],
      "invalid_tool_calls": []
    }
  ]
}
